In [ ]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai gradio numpy


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Day-9   Embeddings.pdf to Day-9   Embeddings (1).pdf


In [ ]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

import os
from getpass import getpass


In [ ]:
import sqlite3
import time
import gradio as gr
import numpy as np


In [ ]:
def init_db():
    conn = sqlite3.connect("rag_memory.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            session_id TEXT,
            user_message TEXT,
            bot_message TEXT,
            citations TEXT,
            confidence REAL,
            response_time REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()


In [ ]:
init_db()


In [ ]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")


Enter your Mistral API Key: ··········


In [ ]:
loader = PyPDFLoader("/content/Day-9   Embeddings.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = splitter.split_documents(docs)

embeddings = MistralAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)


ERROR:langchain_mistralai.embeddings:An error occurred with MistralAI
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_sync/connection.py", line 103, in handle_request
    return self._connection.handle_request(request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/d

LocalProtocolError: Illegal header value b'Bearer b2LzEbli3YqGfe8J3qOZ5hLpXcRefn2o '

In [ ]:
from langchain_mistralai.chat_models import ChatMistralAI


llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0
)


In [ ]:
query_rewrite_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    Rewrite the following query into 3 semantically different search queries:

    Original query: {query}
    """
)


In [ ]:
def hybrid_retrieve(query, k=5):
    docs_scores = vectorstore.similarity_search_with_score(query, k=k)

    # Always keep top-k, but track scores
    docs = []
    scores = []

    for doc, score in docs_scores:
        docs.append(doc)
        scores.append(score)

    return docs, scores


In [ ]:
def build_context(docs):
    if not docs:
        return "No relevant context found in the knowledge base."

    return "\n\n".join(
        [f"[Page {d.metadata.get('page', '?')}] {d.page_content}" for d in docs]
    )


In [ ]:
rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Answer the question ONLY using the context below.
    Cite sources using [source:page].

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)


In [ ]:
def confidence_score(scores):
    if not scores:
        return 0.0

    # Normalize FAISS distances
    avg_score = sum(scores) / len(scores)
    conf = max(0.0, min(1.0, 1 - avg_score))
    return round(conf, 2)


In [ ]:
def save_chat(session_id, user, bot, citations, confidence, response_time):
    conn = sqlite3.connect("rag_memory.db")
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            session_id TEXT,
            user_message TEXT,
            bot_message TEXT,
            citations TEXT,
            confidence REAL,
            response_time REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)

    cursor.execute("""
        INSERT INTO chat_history
        (session_id, user_message, bot_message, citations, confidence, response_time)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (session_id, user, bot, citations, confidence, response_time))

    conn.commit()
    conn.close()


In [ ]:
conn = sqlite3.connect("rag_memory.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.close()


In [ ]:
def chat_fn(message, session_id):
    try:
        start = time.time()

        docs, scores = hybrid_retrieve(message)
        context = build_context(docs)

        response = llm.invoke(
            rag_prompt.format(
                context=context,
                question=message
            )
        )

        conf = confidence_score(scores)

        citations = ", ".join(
            [f"page {d.metadata.get('page', '?')}" for d in docs[:3]]
        ) or "N/A"

        elapsed = round(time.time() - start, 2)

        save_chat(
            session_id=session_id,
            user=message,
            bot=response.content,
            citations=citations,
            confidence=conf,
            response_time=elapsed
        )

        footer = f"""
---
**Confidence:** `{conf}`
**Sources:** {citations}
**Response Time:** `{elapsed}s`
"""

        return response.content + footer

    except Exception as e:
        return f"❌ Error occurred: `{str(e)}`"


In [ ]:
gr.Interface(
    fn=chat_fn,
    inputs=[
        gr.Textbox(label="A vidamga sahayapadagalanu boss"),
        gr.Textbox(value="session_1", label="Session ID")
    ],
    outputs=gr.Markdown(),
    title="Smart RAG Assistant",
    description="Persistent, Trustworthy, Conversational RAG"
).launch(share=True, debug=True)
css = """
.gradio-container {
    background: #0f172a;
    color: white;
}
"""

